In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import datetime
from tensorflow.python.ops import rnn, rnn_cell
from yahoo_finance import Share

In [2]:
day_len = 9
H = 256 # number of hidden layer neurons
D = 9 # input dimensionality
C = 2 # ouput classes 
n_input = 3 # MNIST data input (img shape: 2*2)
n_steps = 3 # timesteps

In [3]:
# get stock from yahoo-finance
stock = Share('2330.TW')
today = datetime.date.today()
stock_data = stock.get_historical('2014-01-01', str(today))
stock_data.reverse()
print len(stock_data)

755


In [4]:
# delete the zero-volumn data
i = 0
while( i < len(stock_data)):
    if (int(stock_data[i].get('Volume')) <= 0):
        stock_data.remove(stock_data[i])
        i = -1
    i += 1
print 'data without zero volumn', len(stock_data)

data without zero volumn 706


In [5]:
# training and testing data
data_X = np.zeros((len(stock_data)-day_len,day_len), dtype=np.float)
data_Y = np.zeros((len(stock_data)-day_len,2), dtype = np.float)
data_tag = []

for i in xrange(0, len(data_X)):
    for j in xrange(0, day_len):
        data_X[i,j] = float(stock_data[i+j].get('Close')) - float(stock_data[i].get('Close'))
    if (i < len(data_X)-1):
        tmp = float(stock_data[i+day_len+1].get('Close')) - float(stock_data[i+day_len].get('Close'))
    if tmp < 0 :
        data_Y[i,0] = tmp
        data_tag.append('fall')
    else :
        data_Y[i,1] = tmp
        data_tag.append('rise')
    
len_train = int(len(data_X) * 0.8)
len_test = len(data_X) - len_train

train_X = data_X[0:len_train]
train_Y = data_Y[0:len_train]
test_X = data_X[len_train::]
test_Y = data_Y[len_train::]
train_tag = data_tag[0:len_train]
test_tag = data_tag[len_train::]

In [6]:
ct_rise = 0
ct_fall = 0

for i in xrange(0, len(train_tag)):
    if train_tag[i]=='rise':
        ct_rise += 1
    else:
        ct_fall += 1

print ct_rise, ct_fall

329 228


In [7]:
test_Y, test_tag, train_X.shape

(array([[ 0. ,  0.5],
        [ 0. ,  0.5],
        [-0.5,  0. ],
        [-0.5,  0. ],
        [ 0. ,  0. ],
        [-1. ,  0. ],
        [-2. ,  0. ],
        [ 0. ,  0.5],
        [ 0. ,  2.5],
        [ 0. ,  0. ],
        [ 0. ,  0. ],
        [ 0. ,  1.5],
        [ 0. ,  5.5],
        [-3. ,  0. ],
        [ 0. ,  3.5],
        [ 0. ,  0.5],
        [ 0. ,  1.5],
        [ 0. ,  0.5],
        [-0.5,  0. ],
        [ 0. ,  2.5],
        [ 0. ,  0. ],
        [ 0. ,  1.5],
        [ 0. ,  0.5],
        [ 0. ,  1. ],
        [ 0. ,  3.5],
        [-3.5,  0. ],
        [ 0. ,  0. ],
        [ 0. ,  1. ],
        [-2.5,  0. ],
        [ 0. ,  2.5],
        [ 0. ,  0. ],
        [ 0. ,  2.5],
        [ 0. ,  0.5],
        [-2. ,  0. ],
        [-5. ,  0. ],
        [-3.5,  0. ],
        [ 0. ,  1.5],
        [ 0. ,  2. ],
        [ 0. ,  3.5],
        [ 0. ,  3.5],
        [ 0. ,  0. ],
        [-2. ,  0. ],
        [-2.5,  0. ],
        [ 0. ,  3. ],
        [ 0. ,  5.5],
        [ 

In [8]:
# LSTM
# placeholder
input_x = tf.placeholder(tf.float32, [None,D], name="input_x")
input_y = tf.placeholder(tf.float32, [None,C], name="input_y")

# weights and biases
w1 = tf.get_variable("W1", shape=[H, C], initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.constant(0.01, shape = [C], name = "B1"))

input_x1 = tf.reshape(input_x, [-1,n_steps, n_input])
input_x2 = tf.transpose(input_x1, [1, 0, 2])
input_x3 = tf.reshape(input_x2, [-1, n_input])
input_x4 = tf.split(0, n_steps, input_x3)

lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(H, forget_bias=1.0)
outputs, states = rnn.rnn(lstm_cell, input_x4, dtype=tf.float32)
score = tf.matmul(outputs[-1], w1) + b1

loss = tf.reduce_mean(tf.reduce_sum(tf.square(score-input_y),reduction_indices=[1]))
optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss) 

In [9]:
init = tf.initialize_all_variables()

In [10]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
      
    for i in range(5000):
        # training
        sess.run(optimizer, feed_dict={input_x: train_X, input_y: train_Y})
        
        if i % 50 == 0:

            pred_tag = []
            print i
            print 'loss:', sess.run(loss, feed_dict={input_x:train_X, input_y: train_Y})

            train_prediction = sess.run(score, feed_dict={input_x: train_X})
            test_prediction = sess.run(score, feed_dict={input_x: test_X})
            for j in xrange(0, len(test_prediction)):
                if test_prediction[j,0]*(-1) - test_prediction[j,1] > 0:
                    pred_tag.append('fall')
                else:
                    pred_tag.append('rise')
                #print pred_tag

0
loss: 4.02887
50
loss: 2.82881
100
loss: 2.57816
150
loss: 2.01942
200
loss: 1.30348
250
loss: 0.666995
300
loss: 0.293892
350
loss: 0.120488
400
loss: 0.0475064
450
loss: 0.0209994
500
loss: 0.0108714
550
loss: 0.00758151
600
loss: 0.00350594
650
loss: 0.00199105
700
loss: 0.00143946
750
loss: 0.000724097
800
loss: 0.000498075
850
loss: 0.00027257
900
loss: 0.000155101
950
loss: 0.000403944
1000
loss: 5.51167e-05
1050
loss: 0.000756569
1100
loss: 2.84997e-05
1150
loss: 2.08398e-05
1200
loss: 0.000184886
1250
loss: 3.80556e-06
1300
loss: 1.9622e-06
1350
loss: 0.00551419
1400
loss: 1.07169e-05
1450
loss: 6.57851e-07
1500
loss: 0.00436723
1550
loss: 1.08703e-05
1600
loss: 3.50566e-07
1650
loss: 1.29057e-07
1700
loss: 5.78864e-08
1750
loss: 3.37759e-07
1800
loss: 0.000187835
1850
loss: 1.2272e-06
1900
loss: 0.000256815
1950
loss: 1.88757e-05
2000
loss: 2.12743e-07
2050
loss: 3.69059e-08
2100
loss: 0.000243613
2150
loss: 7.40312e-05
2200
loss: 2.46115e-07
2250
loss: 1.19466e-06
2300
loss

In [11]:
print pred_tag
print test_tag

['rise', 'fall', 'fall', 'rise', 'rise', 'rise', 'rise', 'rise', 'fall', 'fall', 'fall', 'fall', 'rise', 'rise', 'rise', 'rise', 'rise', 'rise', 'fall', 'rise', 'rise', 'rise', 'rise', 'fall', 'rise', 'rise', 'fall', 'rise', 'fall', 'rise', 'rise', 'fall', 'fall', 'rise', 'fall', 'rise', 'fall', 'fall', 'rise', 'rise', 'fall', 'rise', 'fall', 'rise', 'fall', 'rise', 'fall', 'fall', 'rise', 'rise', 'fall', 'fall', 'fall', 'fall', 'fall', 'fall', 'rise', 'fall', 'rise', 'rise', 'rise', 'rise', 'rise', 'fall', 'rise', 'rise', 'fall', 'fall', 'rise', 'rise', 'fall', 'fall', 'fall', 'rise', 'fall', 'rise', 'rise', 'fall', 'fall', 'rise', 'rise', 'rise', 'rise', 'fall', 'fall', 'rise', 'rise', 'fall', 'fall', 'rise', 'fall', 'fall', 'rise', 'rise', 'rise', 'rise', 'rise', 'fall', 'rise', 'fall', 'rise', 'fall', 'fall', 'rise', 'rise', 'fall', 'rise', 'fall', 'rise', 'rise', 'fall', 'rise', 'rise', 'rise', 'fall', 'rise', 'fall', 'rise', 'rise', 'rise', 'fall', 'fall', 'fall', 'fall', 'fall',

In [12]:
# confusion matrix
tp = 0
fp = 0
fn = 0
tn = 0
for i in xrange (0, len(pred_tag)):
    if pred_tag[i] == test_tag[i] and pred_tag[i] == 'rise':
        tp = tp + 1
    elif test_tag[i] == 'fall' and pred_tag[i] == 'rise':
        fp = fp + 1
    elif test_tag[i] == 'rise' and pred_tag[i] == 'fall':
        fn = fn + 1
    else:
        tn = tn + 1
print 'Accuracy', float(tp + tn) / len(pred_tag)
print '(rise)Precision Rate', float(tp) / float(tp + fp)
print '(fall)Precision Rate', float(tn) / float(tn + fn)
#print 'Recall Rate', float(tp) / float(tp + fn)

Accuracy 0.542857142857
(rise)Precision Rate 0.6375
(fall)Precision Rate 0.416666666667
